# Web-scraping: сбор данных из баз данных и интернет-источников

*Алла Тамбовцева, НИУ ВШЭ*

## Семинар 2. Парсинг с библиотекой BeautifulSoup

Импортируем необходимые библиотеки и функции:

In [ ]:
import requests
from bs4 import BeautifulSoup

Подключаемся к главной странице сайта [nplus1.ru](https://nplus1.ru/). 

In [ ]:
page = requests.get("https://nplus1.ru/")
page

Посмотрим на вид запроса к странице – вызовем атрибут `.raw`:

In [ ]:
page.raw

***

### Задание 1

Запросите атрибуты `url` и `text` и сохраните их в переменные `p_url` и `p_text`. 
***

Теперь подадим исходный код страницы (HTML) на вход функции `BeautifulSoup()`, чтобы превратить его в объект, по которому будет удобно искать информацию по тэгам:

In [ ]:
soup = BeautifulSoup(page.text)
soup

In [ ]:
soup

Для примера выполним поиск по какому-нибудь тэгу с помощью метода `.find_all()`. Например, найдём все заголовки третьего уровня:

In [ ]:
soup.find_all("h3")

Каждый элемент полученного списка – объект типа «элемент beautifulsoup», в который вложено некоторое содержимое, например, текст или новый код на HTML.

***
### Задание 2

Сохраните полученные выше результаты в список `h3`. Запросите тип первого элемента списка с помощью функции `type()`.

### Задание 3

Найдите все ссылки на странице и сохраните их в список `raw_links`.
*** 

Возьмём ссылку на рубрику *Астрономия* и посмотрим не неё, она пятая в полученном списке:

In [ ]:
raw_links[4]

С самим тэгом работать не очень интересно, нас интересует его содержимое. Запросим текст внутри тэга:

In [ ]:
raw_links[4].text

А теперь извлечём саму ссылку – значение атрибута `href`:

In [ ]:
raw_links[4]["href"]

Или так:

In [ ]:
raw_links[4].get("href")

Если бы мы захотели вывести все ссылки на экран, нам понадобился бы цикл `for`:

In [ ]:
for link in raw_links:
    print(link["href"])

***

### Задание 4

Для дальнейшей работы нам нужны только ссылки на новости, то есть те ссылки, которые начинаются с `/news`. Напишите код, который извлекает из списка `raw_links` только те тэги, которые содержат ссылки на новости, и сохраняет в список `news` сами ссылки в виде текста, без тэгов.


### Задание 5

Ссылки на новости в списке `news` – относительные, по ним нельзя сразу перейти на страницу новости, а значит, нельзя передать Python для дальнейшей работы. Сделайте ссылки абсолютными – доклейте к каждой ссылке в `news` ссылку на главную страницу сайта `https://nplus1.ru` и сохраните полученные результаты в список `links_full`.

***


Теперь выберем первую ссылку и напишем код, который будет сгружать информацию о новости по её ссылке. Так как все страницы с новостями на этом сайте строятся по единой схеме, если мы научимся выгружать данные по одной новости, мы сможем повторить это для всех новостей!

In [ ]:
my_link = links_full[0]

Подключимся к странице этой новости, выгрузим её исходный код и превратим в объект `BeautifulSoup`:

In [ ]:
my_page = requests.get(my_link)
my_soup = BeautifulSoup(my_page.text)

Если мы посмотрим на исходный код страницы, мы заметим, что общая информация по новости хранится в тэгах `<meta>`:

In [ ]:
my_soup.find_all("meta")

Как выбрать только те части, которые нам могут быть интересны? Например, части HTML с автором статьи, датой её публикации, заголовком и кратким содержанием? Выполнить более точный поиск с учётом конкретных атрибутов и их значений. Например, мы видим, что имя автора находится в тэге `<meta>` с атрибутом `name`, равным `mediator_author`:

In [ ]:
my_soup.find_all("meta", {"name" : "mediator_author"})

Отлично, мы вышли на автора статьи! Только хотелось бы получить его имя в виде «чистого» текста. Извлечём из полученного списка один единственный элемент и заберём из него значение атрибута `content` (вспомните про работу с `href` ранее):

In [ ]:
my_soup.find_all("meta", {"name" : "mediator_author"})[0].get("content")

In [ ]:
author = my_soup.find_all("meta", 
                 {"name" : "mediator_author"})[0].get("content")

***
### Задание 6

Аналогичным образом извлеките дату публикации новости, заголовок и краткое содержание (описание) новости и сохраните их в переменные `date`, `title`, `desc` соответственно. 

***

Какие ещё характеристики новости нам могут пригодиться (сам текст пока не трогаем)? Время публикации, рубрики и сложность новости. Если пролистаем исходный код до начала самой новости, мы обнаружим перед текстом три таблицы, три абзаца `<p>` с классом `table`. Все они находятся в разделе `<div>` с классом `tables`. 

***
### Задание 7

Извлеките время публикации новости, рубрики и сложность новости и сохраните их в переменные `time`, `rubs`,  `diffc` соответственно.
***